## Understanding tensorflow 

##### Elements of tensorflow:

- Manage Tensorflow program using `tf.Graph`.
- Manage Tensorflow runtime using `tf.Session`

Thus steps to a tensorflow approach are:

- Build **computational graph**. (`tf.Graph`)
- Run computational graph. (`tf.Session`)

#### What is a computational graph?

It's a series of tensorflow operations through which tensors flow. This is arranged into graph.

##### Elements of computational graph:

Every graph have two basic features: `node` and `edges`. 

- `tf.Operation`: This is also called **"ops"**. Ops describe calculation that consume tensor and produce tensor. *They are     nodes of the graph.* 
- `tf.Tensor`: Values that flow through graph. They don't have values but just handles to elements. They are like the pipe through which water flow. *They are edges of the graph.*

In [1]:
import tensorflow as tf

import numpy as np

### Building a simple computational graph

In [2]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0)

> 1) The values 3.0 and 4.0 are 0D Tensor(scalar) to the graph. <br>
> 2) `a` and `b` are the object of `tf.Tensor`. <br>
> 3) In `tf.constant` `constant` is the `ops` which is consuming 3.0 or 4.0 and resturning `tf.Tensor` as a or b.

In [3]:
print(a)
print(b)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)


> 1) `Const` is the operation. <br> 2) The default `dtype` is `float32`. <br> 3) The shape here is null because input tensor is 0D Tensor.<br> 4) Tensors are named after the operation that produce them. <br> 5) Printing them doesnot output the value since at this point we are building the computational graph. 

In [4]:
total = a + b

print(total)

Tensor("add:0", shape=(), dtype=float32)


##### Thus every tf ops takes in tensor and produce tensor. 

### Tensorboard

Tensorboard helps us visualize the computational graph. 

To do so first the computatinal graph is to be saved. When we start building the computational graph using Tensorflow, tf makes that graph as the default graph and we can access the same as shown below.

In [5]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())
writer.flush()

![graph](./graph_run=.png)

### Runtime/Session

To evaluate tensor we need to initialize `tf.Session` also called as **session**. 
Session runs tensorflow `ops`.

In [6]:
sess = tf.Session()

In [7]:
sess.run(total)

7.0

> When `sess.run(ops/output node)` is run tensorflow backtracks through the graph and runs all the nodes that eventually feed input to the requested output node. 

###### During `sess.run` any `tf.Tensor` will have single value/vector. If a `tf.Tensor` is called more than once in single runtime, every time it will contain similar value.

In [8]:
vec = tf.random_uniform((3,))

out1 = vec+1
out2 = vec+2

In [9]:
print(sess.run(vec))
print(sess.run(vec))

# sess.run can take in multiple tensors at once. 
print(sess.run((out1, out2)))

[0.7339653 0.7411051 0.138363 ]
[0.6313909  0.95397556 0.17467153]
(array([1.874285 , 1.4563882, 1.9743004], dtype=float32), array([2.874285 , 2.4563882, 2.9743004], dtype=float32))


> We can see that `vec`, when evaluated twice, gave different random results as expected. But the same when called once by `out1` and once by `out2`, evaluated single vector for `vec`. We can understand this more by looking at the computational graph.

In [10]:
tf.reset_default_graph()

vec = tf.random_uniform((3,))

out1 = vec+1
out2 = vec+2

In [14]:
writer.add_graph(tf.get_default_graph())
writer.flush()

![graph2](graph_run1=.png)

> Thus when `sess.run` is called tensorflow backtrack the graph and start by evaluating `vec`. The output of vec is input to `add` ops of out1 and out2. Thus it gives a single value. 

###### Additional Points

Some TensorFlow functions return `tf.Operations` instead of `tf.Tensors`. The result of calling run on an `Operation` is `None`. *You run an operation to cause a side-effect, not to retrieve a value*. Examples of this include the initialization, and training ops demonstrated later.

### Feeding

The above graphs were kinda boring and redundant as they always produce same values. To train something we need external data which can be provided to the graph using `placeholder`. Placeholder is like the arguments in a function. We first define the shape and dtype of placeholder and while running the graph we feed in the data using `feed_dict`.

In [15]:
tf.reset_default_graph()

In [16]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

z = x + y

In [19]:
with tf.Session() as sess:
    print(sess.run(z, feed_dict={x: 3, y: 10}))
    print(sess.run(z, feed_dict={x:[1,3], y:[20,40]}))

13.0
[21. 43.]


In [21]:
writer.add_graph(tf.get_default_graph())
writer.flush()

![graph 3](graph_run2=.png)

### Datasets

Preferred method to stream data into a graph: `tf.data`

In [24]:
my_data = [[1,2,3],
           [10,20,30],
           [100,200,300]]

In [25]:
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

In [29]:
with tf.Session() as sess:    
    while True:
        try:
            print(sess.run(next_item))
        except tf.errors.OutOfRangeError:
            break

[1 2 3]
[10 20 30]
[100 200 300]


In [31]:
with tf.Session() as sess:
    r = tf.random_normal([10,3])
    dataset = tf.data.Dataset.from_tensor_slices(r)
    iterator = dataset.make_initializable_iterator()
    next_row = iterator.get_next()

    sess.run(iterator.initializer)
    while True:
      try:
        print(sess.run(next_row))
      except tf.errors.OutOfRangeError:
        break

[ 0.37527117 -1.1918255   0.9921601 ]
[0.07526138 0.04971177 0.93111634]
[-0.79718375  0.5649017   0.3902759 ]
[-1.467924   -0.11968345 -1.4690474 ]
[ 0.15523027 -0.5143915   0.06215619]
[-1.3118522   2.8252668  -0.75565743]
[ 0.6310937 -0.5308934  0.2054451]
[-0.18024008  0.13567962 -0.66969615]
[-0.91613305 -0.07085349 -0.17927584]
[-0.7193543  -0.48434746  0.21208327]


### Layers

They are amalgamation of both Tensors and Operations. In a graph we want to modify some parameters so that we can learn some insight with the given same input.

In [33]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

#### Why initialise?

The layers as mentioned have parameters which update when we train the graph. Thus these parameters need to be initialized.

In [35]:
init = tf.global_variables_initializer()

In [38]:
with tf.Session() as sess:
    sess.run(init)
    
    print(sess.run(y, feed_dict={x:[[1, 2, 3],[4, 5, 6]]}))

[[-3.199371]
 [-7.60085 ]]
